In [1]:
using CSV 
using DataFrames, Random
using Clustering

# set random seed
Random.seed!(1234)

TaskLocalRNG()

In [2]:
df_fits =  CSV.read("./Results/Synthetic_population_fits.csv", DataFrame)

Row,ID,Meal Test,Health status,k1,k5,k6,k8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,0.0,0.0104072,0.0173463,4.40951,7.354
2,1.0,2.0,0.0,0.0104362,0.0155601,5.10032,7.57521
3,1.0,3.0,0.0,0.0105299,0.0131662,5.54484,7.59195
4,1.0,4.0,0.0,0.0104272,0.0116063,5.3463,7.08974
5,1.0,5.0,0.0,0.0103704,0.0109003,4.90771,6.83445
6,1.0,6.0,0.0,0.0104962,0.00845506,3.49573,7.08165
7,1.0,7.0,0.0,0.0104589,0.0058106,3.53309,7.01873
8,2.0,1.0,1.0,0.01038,0.0164254,5.00868,7.44119
9,2.0,2.0,1.0,0.0105607,0.0166048,4.63033,7.29184


In [3]:
# Rename the columns to be used in unstack
df_k5s = rename(df_fits,  Symbol("Meal Test") => :parameter, :k5 => :value)
# Get the unique IDs and Health status
df_health_status = unique(df_fits[:, [:ID, Symbol("Health status")]])

# Unstack the DataFrame
df_unstacked = unstack(df_k5s, :ID,  :parameter, :value)
# Join the "Health status" DataFrame with the unstacked DataFrame
df_unstacked = rightjoin(df_unstacked, df_health_status, on = :ID)
#R = kmeans(k5_data,2)


# Get the data for the clustering
k5_data = df_unstacked[:, 2:end-1]
k5_data = Matrix{Float64}(k5_data)	
println(k5_data)
# Perform the clustering
R = kmeans(k5_data', 2)
# Find the missing values

a = assignments(R) # get the assignments of points to clusters
c = counts(R) # get the cluster sizes





[0.01734628159166582 0.015560100420608174 0.013166201354272841 0.011606303168313244 0.010900300867591091 0.008455061919656861 0.005810596565899253; 0.016425356382514283 0.016604790918781306 0.017357175701227145 0.016594029766924586 0.01710870605433626 0.018391312959718516 0.01691323111485573; 0.017741123604684116 0.014733305573794325 0.014873538264214826 0.011703281017250964 0.010104662722040225 0.006863866196773451 0.006936286591068973; 0.01723963007312067 0.016632303701400324 0.014797975203949761 0.01640993297974588 0.015501062401041559 0.016842248548831824 0.01510189067109356; 0.01672105229640699 0.014722334129898672 0.01246051447964216 0.011179392926622053 0.009823347589085462 0.007238445265190115 0.006778085045786041; 0.01957763337525204 0.019453438590055418 0.018053612846445362 0.02024442443677467 0.016467483861426227 0.020081009291218737 0.018658787238648687; 0.01987966268820765 0.017302605701466866 0.013159940780327927 0.012456475948762622 0.009653940957124109 0.008082935826705

2-element Vector{Int64}:
 5
 5

In [7]:
using MultivariateStats, Plots

# Perform PCA
M = fit(PCA, k5_data'; maxoutdim=2)

# Project the data onto the principal components
Yte = MultivariateStats.transform(M, k5_data')

# Get the variance explained by each component
ve = round.(M.prinvars / sum(M.prinvars);digits=4)
ve1 = ve[1]
ve2 = ve[2]

# Create a scatter plot of the projected data, colored by cluster assignment
scatter(Yte[1, :], Yte[2, :], color=a, legend=false, xlabel="PC1 ($ve1%)" , ylabel="PC2 ($ve2%)", title="PCA parameter k5")

savefig("k5_clusters.png")


"c:\\Users\\20192809\\Documents\\GitHub\\EDES-thesis\\k5_clusters.png"

In [5]:
# implement a elbow plot
using Clustering
using Plots

# Create a vector to store the sum of squared errors
sse = Float64[]
# Create a vector to store the number of clusters
k = 1:10
# Loop over the number of clusters
for i in k
    # Perform the clustering
    R = kmeans(k5_data', i)
    # Get the sum of squared errors
    push!(sse, sum(R.dists))
end

# Create a plot of the sum of squared errors
plot(k, sse, xlabel="Number of clusters", ylabel="Sum of squared errors", title="Elbow plot for k5")


ErrorException: type KmeansResult has no field dists